# Paralelni algoritmi - treći projekat

1. [5 bodova] Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)
2. [5 bodova] Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. [7 bodova] Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)
4. [8 bodova] Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 5.5MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620705 sha256=477c596ebf32abeda41ce351f77ec196bddabe2fe4487d1a00a6d615a82e2897
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.4-py2.py3-none-any.whl size=59109 sha256=207cbd1931e468881692ba5de71fbd8393b8aafd3cd94c51c356e967830f8650
  Stored in directory: /root/.cache/pip/wheels/6f/2a/6e/0b210d900f2b6caca133395d2fa153d8a9c03ad42c3c5d55bf
Successfully built pycuda pytools


In [ ]:
!nvidia-smi

Thu Dec 24 20:46:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1. [5 bodova] Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

mod = SourceModule("""
      __global__ void oneBlockMulti(float *a, float *b, float *c, int kolA, int kolB) {

        int indexC = threadIdx.x + threadIdx.y*kolB;
        int indexA = threadIdx.y*kolA;
        int indexB = threadIdx.x;
        float fValue = 0;
        int i = 0;
        for (i; i < kolA; i++) {
          float aElement = a[indexA + i];
          float bElement = b[indexB];
          fValue += aElement * bElement;
          indexB += kolB;
        }
        c[indexC] = fValue;
      }
      """)

a = np.random.randn(20,15).astype(dtype=np.float32)
b = np.random.randn(15,10).astype(dtype=np.float32)

res_check = a.dot(b) # Rezultat za proveru
c = np.zeros_like(res_check)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu, c)

func = mod.get_function("oneBlockMulti")
func(a_gpu, b_gpu, c_gpu, np.int32(a.shape[1]), np.int32(b.shape[1]), block=(np.int(b.shape[1]), np.int(a.shape[0]), 1), grid=(1, 1, 1))

cuda.memcpy_dtoh(c, c_gpu)

if (c==res_check).all():
  print('Jednake matrice')
  print(c)
else:
  print('Nejednake!')


Jednake matrice
[[-3.5522156   9.903658   -0.9676281   4.330498    3.0890806   3.4862545
  -4.4934382   1.0412718  -6.4077115  -1.5505251 ]
 [ 2.913807    4.2815156   1.9980567  -0.48132947 -4.2101917   0.06444133
  -2.4775846   4.169553   -0.7243547  -1.4836429 ]
 [ 5.6167545   2.7104738   2.4914436  -1.2026819  -3.54126     3.3654313
  -1.9746261   3.583166   -1.180192    0.97441244]
 [ 1.4959023   2.6264045  -3.6920369   5.4260235   5.2990913   5.6880565
  -4.1103992   2.3620925  -0.45145452  1.7505283 ]
 [ 1.3784122   3.2323766  -1.487074    5.4414415   2.9284792  -1.0043005
  -2.9614975  -1.6788462   1.7889653   4.1380525 ]
 [ 8.031756    2.9292958   0.15897255 -0.31539473 -1.5239409  -0.40969536
  -1.62107     4.3733873  -0.52170813  0.75442123]
 [ 3.1193721   0.6397917  -3.2804976   2.7520983  -1.751309   -1.5874898
  -3.6179986  -3.5494227   3.776781   -4.5270977 ]
 [ 3.7440426  -0.48518825  4.505292   -0.78590703 -3.8492115   1.2985326
  -0.78580767 -0.33166093  0.83081084 -0.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(10): warning: expression has no effect




2. [5 bodova] Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.

In [155]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

mod = SourceModule("""

      __global__ void multiBlockMultiply(float *a, float *b, float *c, int kolA, int kolB, int redA, int redB) {

        long indexC = threadIdx.x + blockDim.x * blockIdx.x + threadIdx.y * kolB + blockDim.y * blockIdx.y * kolB;
        long indexA = (threadIdx.y + blockDim.y * blockIdx.y) * kolA;
        long indexB = blockDim.x * blockIdx.x + threadIdx.x;

        if ((indexB < kolB) && (threadIdx.y + blockDim.y * blockIdx.y < redA)) {

          float fValue = 0;
          
          for (int i = 0; i < kolA; i++) {
            float aElement = a[indexA + i];
            float bElement = b[indexB];
            fValue += aElement * bElement;
            indexB += kolB;
          }

          c[indexC] = fValue;

        }

      }

      """)

a = np.random.randn(64, 64).astype(dtype=np.float32)
b = np.random.randn(64, 256).astype(dtype=np.float32)
a = 100*a
b = 100*b
a = np.round(a, 1)
b = np.round(b, 1)

res_check = a.dot(b)
res_check = np.round(res_check, 1)
c = np.zeros_like(res_check)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu, c)

func = mod.get_function("multiBlockMultiply")
func(a_gpu, b_gpu, c_gpu, 
     np.int32(a.shape[1]), np.int32(b.shape[1]), 
     np.int32(a.shape[0]), np.int32(b.shape[0]), 
     block=(32, 32, 1), 
     grid=(math.ceil(b.shape[1]/32),
           math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c, c_gpu)
c = np.round(c, 1)

print('res_check**************************************************')
print(res_check)
print('c**********************************************************')
print(c)


if (c==res_check).all():
  print("Jednake")
  print(c)
else:
  print("Nisu jednake")
  print(c-res_check)

res_check**************************************************
[[ -27923.7   -1010.8  -69580.2 ...  -91267.8 -143228.5   69323.7]
 [  48605.2   68017.7   26686.1 ...   31133.4 -148783.7   30821.7]
 [  73308.   -10169.9  -32331.5 ... -114710.4   70981.4   31109.6]
 ...
 [  60242.6  -30405.2 -139434.4 ...  -55799.8  151600.2 -186146.1]
 [-119911.4  -94329.8   28492.1 ...   -5379.3    8451.9  -40511.1]
 [  47623.3  -91119.3   60742.8 ...   14518.9  -10603.4   43904. ]]
c**********************************************************
[[ -27923.7   -1010.8  -69580.2 ...  -91267.8 -143228.5   69323.6]
 [  48605.2   68017.7   26686.1 ...   31133.4 -148783.7   30821.7]
 [  73308.   -10169.9  -32331.5 ... -114710.4   70981.4   31109.6]
 ...
 [  60242.6  -30405.2 -139434.4 ...  -55799.8  151600.2 -186146.2]
 [-119911.3  -94329.8   28492.1 ...   -5379.3    8451.9  -40511.1]
 [  47623.3  -91119.3   60742.8 ...   14518.9  -10603.4   43904. ]]
Nisu jednake
[[ 0.         0.         0.        ...  0.         

3. [7 bodova] Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memoriju, a potom sve čitaju iz deljene memorije)

In [156]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math


mod = SourceModule("""
    __global__ void multiply(float *a, float *b, float *c, int n, int m, int k, int iter) {
        float sol = 0;
        __shared__ float dataA[1024];
        __shared__ float dataB[1024];

        for(int i=0; i<iter; i++) {
            long ida = blockIdx.y*blockDim.y*m + threadIdx.y*m + i*blockDim.x + threadIdx.x;
            long idb = i*blockDim.y*k + threadIdx.y*k + blockIdx.x * blockDim.x + threadIdx.x;

            if ((i*blockDim.x + threadIdx.x < m) && (blockIdx.y*blockDim.y + threadIdx.y < n))
                dataA[threadIdx.x + threadIdx.y*blockDim.x] = a[ida];
            
            if ((blockIdx.x * blockDim.x + threadIdx.x < k) && (i*blockDim.y + threadIdx.y < m))
                dataB[threadIdx.x + threadIdx.y*blockDim.x] = b[idb];

            __syncthreads();

            int dif = m - i*32;
            if (dif > 32)
                dif = 32;
            for(int j=0; j<32; j++) {
                if (j < dif)
                    sol += (dataA[threadIdx.y*blockDim.x + j] * dataB[threadIdx.x + j*blockDim.x]);
            }
            __syncthreads();
        }


        long idc = blockIdx.y*blockDim.y*k + threadIdx.y * k + blockIdx.x * blockDim.x + threadIdx.x;
        if ((blockIdx.x * blockDim.x + threadIdx.x < k) && (blockIdx.y*blockDim.y + threadIdx.y < n))
            c[idc] = sol;
    }
""")

a = np.random.randn(40, 70).astype(dtype=np.float32)
b = np.random.randn(70, 100).astype(dtype=np.float32)
a = 100*a
b = 100*b
a = np.round(a, 1)
b = np.round(b, 1)

res_check = a.dot(b)
res_check = np.round(res_check, 1)
c = np.zeros_like(res_check).astype(dtype=np.float32)
c = np.round(c, 1)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu, c)

func = mod.get_function("multiply")
func(a_gpu, b_gpu, c_gpu, np.int32(c.shape[0]), np.int32(b.shape[0]), np.int32(c.shape[1]), np.int32(math.ceil(b.shape[0]/32)),
     block=(32, 32, 1),
     grid=(math.ceil(c.shape[1]/32), math.ceil(c.shape[0]/32), 1))

cuda.memcpy_dtoh(c, c_gpu)
c = np.round(c, 1)

print('res_check**************************************************')
print(res_check)
print('c**********************************************************')
print(c)

if (c==res_check).all():
  print("Jednake")
  print(c)
else:
  print("Nisu jednake")
  print(c-res_check)

res_check**************************************************
[[-130656.8   57289.2   15643.4 ...  -75679.6   99708.3   63267.5]
 [ -55643.7 -161859.5  -67817.  ...   79874.2  -65483.3   17052.7]
 [ -68607.7  -10756.   -35301.6 ... -139987.    96119.9   45369.3]
 ...
 [ -57547.7   29481.4  -48225.4 ...  -33405.8  -24940.3   -2849.6]
 [  25865.   -59423.3  -10340.9 ... -149667.   -45692.1  -59406.2]
 [  27791.1  -39780.5   33127.9 ...  -87989.2   84758.3   13696.7]]
c**********************************************************
[[-130656.8   57289.2   15643.3 ...  -75679.6   99708.3   63267.5]
 [ -55643.7 -161859.5  -67817.  ...   79874.2  -65483.3   17052.7]
 [ -68607.7  -10756.1  -35301.6 ... -139987.    96119.9   45369.3]
 ...
 [ -57547.8   29481.5  -48225.4 ...  -33405.8  -24940.3   -2849.6]
 [  25865.   -59423.3  -10340.9 ... -149667.   -45692.1  -59406.2]
 [  27791.1  -39780.5   33127.9 ...  -87989.2   84758.3   13696.7]]
Nisu jednake
[[ 0.          0.         -0.10058594 ...  0.      

4. [8 bodova] Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [158]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import math

mod = SourceModule("""

    __global__ void multiplyTransposed(float *a, float *b, float *c, int n, int m, int k) {
        float sol = 0;
        __shared__ float dataA[1024];
        __shared__ float dataB[1024];

        for(int i=0; i<m; i++) {
            long ida = blockIdx.y*blockDim.y*m + threadIdx.y*m + i*blockDim.x + threadIdx.x;
            long idb = blockIdx.x*blockDim.y*m + threadIdx.y*m + i*blockDim.x + threadIdx.x;

            if ((i*blockDim.x + threadIdx.x < m) && (blockIdx.y*blockDim.y + threadIdx.y < n))
                dataA[threadIdx.x + threadIdx.y*blockDim.x] = a[ida];
            
            if ((i*blockDim.x + threadIdx.x < m) && (blockIdx.x*blockDim.y + threadIdx.y < k))
                dataB[threadIdx.x + threadIdx.y*blockDim.x] = b[idb];

            __syncthreads();

            int dif = m - i*32;
            if (dif > 32)
                dif = 32;
            for(int j=0; j<32; j++) {
                if (j < dif)
                    sol += (dataA[threadIdx.y*blockDim.x + j] * dataB[threadIdx.x*blockDim.x + j]);
            }

            __syncthreads();
        }

        long idc = blockIdx.y*blockDim.y*k + threadIdx.y * k + blockIdx.x * blockDim.x + threadIdx.x;
        if ((blockIdx.x * blockDim.x + threadIdx.x < k) && (blockIdx.y*blockDim.y + threadIdx.y < n))
            c[idc] = sol;
    }

""")

a = np.random.randn(200, 64).astype(dtype=np.float32)
b = np.random.randn(300, 64).astype(dtype=np.float32)
a = np.round(a, 1)
b = np.round(b, 1)
c = np.zeros(shape=(a.shape[0], b.shape[0])).astype(dtype=np.float32)
c = np.round(c, 1)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu, c)

bt = np.copy(b)
bt = np.transpose(bt)

res_check = a.dot(bt)
res_check = np.round(res_check, 1)

func = mod.get_function("multiplyTransposed")
func(a_gpu, b_gpu, c_gpu, np.int32(c.shape[0]), np.int32(b.shape[1]), np.int32(c.shape[1]), block=(32, 32, 1), grid=(math.ceil(c.shape[1]/32), math.ceil(c.shape[0]/32), 1))

cuda.memcpy_dtoh(c, c_gpu)
c = np.round(c, 1)

print('res_check**************************************************')
print(res_check)
print('c**********************************************************')
print(c)

if (c==res_check).all():
  print("Jednake")
  print(c)
else:
  print("Nisu jednake")
  print(c-res_check)

res_check**************************************************
[[ -2.8  -1.3  14.9 ...   7.    6.1 -14.4]
 [ -1.1  -2.3  -9.1 ...  -4.2  -6.8  -1.6]
 [  4.8 -14.1  -5.4 ...   0.7 -14.6  -4.3]
 ...
 [-11.3   0.1  -8.4 ...   2.5  -0.5   6.1]
 [ -1.9  10.7  -7.2 ...  -1.2  13.9  -2.4]
 [-10.8  -5.9  11.5 ... -14.3   0.7 -12.3]]
c**********************************************************
[[ -2.8  -1.3  14.9 ...   7.    6.1 -14.4]
 [ -1.1  -2.3  -9.1 ...  -4.2  -6.8  -1.6]
 [  4.8 -14.1  -5.4 ...   0.7 -14.6  -4.3]
 ...
 [-11.3   0.1  -8.4 ...   2.5  -0.5   6.1]
 [ -1.9  10.7  -7.2 ...  -1.2  13.9  -2.4]
 [-10.8  -5.9  11.5 ... -14.3   0.7 -12.3]]
Nisu jednake
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
